In [46]:
#only for testing
def return_question():
    import random
    next_ques=random.choice(["Yooooo!",'Heyyyy!','Weeeee!'])
    pic_link='https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
    more_info_link='https://stackoverflow.com/questions/14076674/using-tkinter-to-open-a-webpage'
    return next_ques,pic_link,more_info_link
#return a random object id and category
def get_question():
    import random
    category = random.choice(['artist','time','material','type','department'])
#     obj_id = random.choice(valid_obj_id)
    info = create_question(category, obj_id)
    pic_link = get_image(obj_id)
    title = get_title(obj_id)
    obj_link = get_url(obj_id)
    return info, pic_link, title, obj_link

#return the desired answer info of the randomly selected art
def get_image(ind):
    return df.iloc[ind]['image_url']

def get_title(ind):
    return df.iloc[ind]['title']

def get_url(ind):
    return df.iloc[ind]['object URL']

def get_artist(ind):
    return df.iloc[ind]['artist Display Name']

def get_time(ind):
    return df.iloc[ind]['century']

def get_material(ind):
    return df.iloc[ind]['medium']

def get_country(ind):
    return df.iloc[ind]['country']

def get_department(ind):
    return df.iloc[ind]['department']

#generate question stem and answer choices
def create_question(category, obj_id):
    choice = list()
    if category is 'artist':
        question = "Who is the artist for this piece of work?"
        correct = obj.get_artist()
        wrong1 = w1.get_artist()
        wrong2 = w2.get_artist()
        wrong3 = w3.get_artist()
    elif category is 'time':
        question = "During which time period was this artwork created?"
        correct = obj.get_time()
        wrong1 = w1.get_time()
        wrong2 = w2.get_time()
        wrong3 = w3.get_time()
    elif category is 'material':
        question = "What kind of main material did this object use?"
        correct = obj.get_material()
        wrong1 = w1.get_material()
        wrong2 = w2.get_material()
        wrong3 = w3.get_material()
    elif category is 'type':
        question = "What type of artwork is this?"
        correct = obj.get_type()
        wrong1 = w1.get_type()
        wrong2 = w2.get_type()
        wrong3 = w3.get_type()
    else: #department
        question = "Which department does this exhibit belong to?"
        correct = obj.get_department()
        wrong1 = w1.get_department()
        wrong2 = w2.get_department()
        wrong3 = w3.get_department()

    choice = [wrong1, wrong2, wrong3, correct]
    return question, choice

#return 4 choices and answer key
def get_choices():
    import random
    from random import shuffle
    ####### Need to be linked to previous function
    #answer_list = get_question()[0][1]
    answer_list = ["wrong1","wrong2","wrong3","correct"]
    index_list = ['A','B','C','D']
    random.shuffle(answer_list)
    answer = dict(zip(index_list,answer_list))
    answer_key = [key for key, value in answer.items() if value == "correct"] #letter w/ correct answer
    answer_list=list(answer.items())
    answer_string_list=[]
    for answer in answer_list: #create list w/ entries like 'A. correct'
        x=answer[0]+'. '+answer[1]
        answer_string_list.append(x)
    return answer_key, answer_string_list

In [47]:
from tkinter import *
from tkinter import scrolledtext
import urllib.request
from PIL import Image, ImageTk
from io import BytesIO

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("700x600")
        root.title("MET-Q")
        self.root = root
        
        #canvas
        canvas_width = 700
        canvas_height = 600
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        self.canvas=canvas
        

        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var,height=2,width=8)
        self.cor_label.config(bg='linen')
        self.var=var
        self.cor_label.grid(row=0,column=8)
        
        #title label
        self.title_label = Label(root, text="The MET Quiz", font = "Times 20 bold")
        self.title_label.config(bg='linen')
        self.title_label.grid(row=0,column=1,columnspan=7)
    
        #record the number of correct answers so far
        self.total = IntVar() 
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var)
        self.score_label.config(bg='linen')
        self.score_label.grid(row=1,column=0,columnspan=3)
        self.already_chosen=0
        
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var)
        self.num_of_ques.config(bg='linen')
        self.num_of_ques.grid(row=1,column=6,columnspan=3)
    
        #question stem
        question_var=StringVar()
        self.Q = Label(root, textvariable=question_var)
        question_var.set(get_question()[0][0])
        self.Q.grid(row=2,columnspan=9)
        self.Q.config(bg='linen')
        self.question_var=question_var
        
        #object title
        obj_title_var = StringVar()
        self.obj_title = Label(root,textvariable=obj_title_var)
        obj_title_var.set(get_question()[2])
        self.obj_title.grid(row=3,columnspan=3)
        self.obj_title.config(bg='linen')
        self.obj_title_var = obj_title_var
        
        #Question picture
        URL=return_question()[1]
        u = urllib.request.urlopen(URL)
        raw_data = u.read()
        u.close()
        image_file = Image.open(BytesIO(raw_data))
        image_file=image_file.resize((200,200),Image.ANTIALIAS)
        photo_image = ImageTk.PhotoImage(image_file)
        self.pic = Label(root,image=photo_image)
        self.pic.config(bg='linen')
        self.pic.grid(row=4,columnspan=9)
        
        #choice buttons
        choices=get_choices()
        self.correct_answer=choices[0][0]
        self.choice_A=StringVar()
        self.choice_A.set(choices[1][0])
        self.choice_B=StringVar()
        self.choice_B.set(choices[1][1])
        self.choice_C=StringVar()
        self.choice_C.set(choices[1][2])
        self.choice_D=StringVar()
        self.choice_D.set(choices[1][3])
        self.A = Button(root, textvariable=self.choice_A,command=lambda: self.check_correct('A',self.correct_answer))
        self.A.grid(row=5,column=2,columnspan=5,pady=5)
        self.B = Button(root, textvariable=self.choice_B,command=lambda: self.check_correct('B',self.correct_answer))
        self.B.grid(row=6,column=2,columnspan=5,pady=(0,5))
        self.C = Button(root, textvariable=self.choice_C,command=lambda: self.check_correct('C',self.correct_answer))
        self.C.grid(row=7,column=2,columnspan=5,pady=(0,5))
        self.D = Button(root, textvariable=self.choice_D,command=lambda: self.check_correct('D',self.correct_answer))
        self.D.grid(row=8,column=2,columnspan=5,pady=(0,5))
        self.next=Button(root,text='Next question',command=lambda:self.next_()).grid(row=8,column=2,columnspan=5)

        #quit button
        self.close_button=Button(root, text="Quit", command=lambda: root.destroy())
        self.close_button.grid(row=10,column=8)
        
        #more information link
        self.more_info_link=return_question()[2]
        self.link=Button(root,text='Know more',command=lambda: self.open_page(self.more_info_link))
        self.link.grid(row=10,column=2,columnspan=5)  
        
        root.configure(background="papaya whip")
        root.mainloop() 

    #add 1 to score if the correct answer is chosen
    def check_correct(self,this_answer,correct_answer):
        if this_answer==correct_answer:
            self.correct()
        else:
            self.incorrect()
                        
    def correct(self):
        self.var.set('Correct')
        if self.already_chosen==0:
            self.total.set(self.total.get() + 1) 
            self.total_var.set('Number of correct answers: '+str(self.total.get()))
            self.already_chosen=1
    
    def incorrect(self):
        self.var.set('Incorrect')
        self.already_chosen=1
        
    def next_(self):
#         next_question=return_question()
        next_question = get_question()
        question = next_question[0][0]
        next_pic = next_question[1]
        next_title = next_question[2]
        self.question_var.set(question)
        self.num.set(self.num.get()+1)
        self.num_var.set('Questions completed: '+str(self.num.get()))
        next_choices=get_choices()
        correctanswer=next_choices[0][0]
        self.correct_answer=correctanswer
        self.choice_A.set(next_choices[1][0])
        self.choice_B.set(next_choices[1][1])
        self.choice_C.set(next_choices[1][2])
        self.choice_D.set(next_choices[1][3])
#         self.more_info_link=return_question()[2]
        self.more_info_link = next_question[3]
        self.already_chosen=0
        
    def open_page(self,link):
        import webbrowser
        webbrowser.open(link)

In [48]:
myGUI = quizGUI()

NameError: name 'obj_id' is not defined